In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
        
def datasplit_new(df,testsize,relovir):
    """
    Firstly we split the dataset into train and test parts.
    
    Then we create the training dataset by picking up the irrelevant tweets from the training 
    split part with only the number of relevant tweet
    
    The relovir variable represents the relative ratio of irrelevant(we usually have more irrelevant so) 
    over relevant number of training examples in the set.
    
    Returns as a (examples,768) np array the representations and the y as (examples,) shaped np array.
    
    Future: we need to be able to reduce zero and one examples accordinglydepending of which 
    
    """
    # make a copied instance of the dataset
    df = df.copy()
    # we split the dataset into train and test subsets
    df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df['reps'], df['relevance'], test_size = testsize)
    # reconstruct training set
    if relovir>0:
        training_set = pd.DataFrame()
        training_set['reps'] = df_X_train.copy()
        training_set['relevance'] = df_y_train.copy()
        training_set.reset_index(drop = True)
        # we split the training dataset by relervance into two DataFrames irr and rel
        grouping = training_set.groupby('relevance')
        group_dict = {}
        for name, group in grouping:
            group_dict[str(name)] = group
        # we find the absolute numbers 
        df_training_irr = group_dict['0'].reset_index(drop = True)
        df_training_rel = group_dict['1'].reset_index(drop = True)
        # based on the relovir parameter pick up the set with the appropriate ratio, here an explanation for the inner if is needed:
        Nrel = len(df_training_rel)
        # print(Nrel)
        Nirr = len(df_training_irr)
        # print(Nirr)
        N = Nrel + Nirr
        if relovir<=1:
            if relovir<Nrel/Nirr:
                relevant_part = df_training_rel.sample(n = int(Nirr*relovir))
                df_training = pd.concat([df_training_irr, relevant_part]).sample(frac=1).reset_index(drop = True)
            else:
                df_training = pd.concat([df_training_irr, df_training_rel]).sample(frac=1).reset_index(drop = True)
        else:
            if relovir<Nrel/Nirr:
                irrelevant_part = df_training_irr.sample(n = int((1/relovir)*Nrel))
                df_training = pd.concat([df_training_rel, irrelevant_part]).sample(frac=1).reset_index(drop = True)
            else:
                df_training = pd.concat([df_training_irr, df_training_rel]).sample(frac=1).reset_index(drop = True)
        df_X_train = df_training['reps'].copy()
        df_y_train = df_training['relevance'].copy()
    else:
        print("relovir can't be negative or zero")
    df_X_train.apply(lambda x: x.reshape(768,))
    df_X_test = df_X_test.copy().apply(lambda x: x.reshape(768,))
    training_set_X = np.vstack(df_X_train)
    test_set_X = np.vstack(df_X_test)
    y_train = np.vstack(df_y_train).reshape(-1)
    y_test = np.vstack(df_y_test).reshape(-1)
    return training_set_X, test_set_X, y_train, y_test

training_set_X, test_set_X, training_set_y, test_set_y = datasplit_new(WF,0.3,relovir = 0.4)
print(f"Το training set X: {type(training_set_X)} με {training_set_X.shape}")
print(f"To test set X: {type(test_set_X)} με {test_set_X.shape}")
print(f"To training set y: {type(training_set_y)} με {training_set_y.shape}")
print(f"To test set y: {type(test_set_y )} με {test_set_y.shape}")